In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

%matplotlib inline

In [2]:
master_df = pd.read_csv('data/master_df_for_modeling.csv')

In [3]:
master_df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,dairy eggs,eggs,202279,prior,3,Friday,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,produce,fresh vegetables,202279,prior,3,Friday,9,8.0
2,2,17794,6,1,Carrots,83,4,produce,fresh vegetables,202279,prior,3,Friday,9,8.0
3,2,9327,3,0,Garlic Powder,104,13,pantry,spices seasonings,202279,prior,3,Friday,9,8.0
4,2,45918,4,1,Coconut Butter,19,13,pantry,oils vinegars,202279,prior,3,Friday,9,8.0


In [4]:
master_df.shape

(32434489, 15)

## Building a Recommender System Using master_Df & cosine similarity

In [5]:
master_df['reordered'].value_counts()

1    19126536
0    13307953
Name: reordered, dtype: int64

In [27]:
# master_reorders = master_df[master_df['reordered'] == 1]

In [7]:
master_df['volume_higher'] = (master_df['product_id'].value_counts().sort_values(ascending=False) > 1)

In [8]:
master_df['volume_higher'].value_counts()

True     49546
False      131
Name: volume_higher, dtype: int64

In [32]:
# reorders = master_reorders[master_reorders['reordered'] == 1]
# reorders.head()
# print(reorders.shape)

(2285465, 16)


In [9]:
high_volume = master_df[master_df['volume_higher'] == True]


In [10]:
high_volume.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'department', 'aisle',
       'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'volume_higher'],
      dtype='object')

In [11]:
high_volume['aisle'].value_counts()

fresh vegetables              5374
fresh fruits                  5285
eggs                          3520
packaged vegetables fruits    2754
yogurt                        2026
                              ... 
first aid                        8
specialty wines champagnes       8
frozen juice                     8
kitchen supplies                 8
eye ear care                     7
Name: aisle, Length: 134, dtype: int64

In [12]:
high_volume_users = high_volume.groupby(['user_id', 'aisle']).size().sort_values(ascending=False).unstack().fillna(0)

In [13]:
high_volume['reordered'].value_counts()

1    30069
0    19477
Name: reordered, dtype: int64

In [14]:
# read in prior_orders_cleaned
prior_orders_clean = pd.read_csv('data/prior_orders_cleaned_df.csv')
prior_orders_clean.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,dairy eggs,eggs
1,22,23341,6,1,Large Grade AA Eggs,86,16,dairy eggs,eggs
2,25,6383,7,1,All Whites 100% Egg Whites,86,16,dairy eggs,eggs
3,26,33120,5,0,Organic Egg Whites,86,16,dairy eggs,eggs
4,55,11520,12,0,Large Alfresco Eggs,86,16,dairy eggs,eggs


In [15]:
orders = pd.read_csv('./data/orders.csv')

In [16]:
orders2 = orders[['order_id', 'user_id']]

In [18]:
# user_orders = reorders.merge(orders2, on='order_id')

In [40]:
# user_orders['high_volume'] = (user_orders['product_id'].value_counts().sort_values(ascending=False) > 1)
# user_orders.shape

(2285465, 18)

In [42]:
# # high_volume = user_orders[user_orders['high_volume'] == True]
# high_volume.shape
# high_volume.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,user_id_x,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,volume_higher,user_id_y,high_volume
1,2,28985,2,1,Michigan Organic Kale,83,4,produce,fresh vegetables,202279,prior,3,Friday,9,8.0,True,202279,True
3,2,45918,4,1,Coconut Butter,19,13,pantry,oils vinegars,202279,prior,3,Friday,9,8.0,True,202279,True
4,2,40141,7,1,Original Unflavored Gelatine Mix,105,13,pantry,doughs gelatins bake mixes,202279,prior,3,Friday,9,8.0,NaN,202279,True
5,2,1819,8,1,All Natural No Stir Creamy Almond Butter,88,13,pantry,spreads,202279,prior,3,Friday,9,8.0,True,202279,True
7,22,28993,4,1,Iceberg Lettuce,83,4,produce,fresh vegetables,30822,prior,25,Monday,8,12.0,True,30822,True


In [43]:
# high_volume['reordered'].value_counts()

1    26380
Name: reordered, dtype: int64

In [19]:
high_volume.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'department', 'aisle',
       'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'volume_higher'],
      dtype='object')

In [20]:
high_volume_users = high_volume.groupby(['user_id', 'aisle']).size().sort_values(ascending=False).unstack().fillna(0)
high_volume_users

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
150,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206104,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
206165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
recommender_cosine = pairwise_distances(high_volume_users, metric='cosine')
recommender_cosine.shape

(3252, 3252)

In [49]:
# recommender_corr = pairwise_distances(high_volume_users, metric='correlation')
# recommender_corr.shape

(4655, 4655)

In [24]:
# recommender_cosine_df = pd.DataFrame(recommender_cosine, columns=high_volume_users.index, index=high_volume_users.index)
# recommender_cosine_df

In [23]:
# recommender_corr_df = pd.DataFrame(recommender_corr, columns=high_volume_users.index, index=high_volume_users.index)
# recommender_corr_df

In [25]:
recommender = cosine_similarity(high_volume_users)

In [26]:
recommender_df = pd.DataFrame(recommender, columns=high_volume_users.index, index=high_volume_users.index)
recommender_df

user_id,7,137,140,150,155,226,396,417,587,593,...,205619,205695,205700,205811,206103,206104,206165,206206,206207,206208
user_id,,,,,,,,,,,,,,,,,,,,,
7,1.000000,0.060634,0.287837,0.194461,0.350070,0.453743,0.253320,0.121268,0.230089,0.171499,...,0.600245,0.375735,0.171499,0.240663,0.545705,0.329073,0.219382,0.121268,0.383482,0.544581
137,0.060634,1.000000,0.029670,0.066815,0.144338,0.200446,0.130558,0.125000,0.316228,0.176777,...,0.088388,0.193649,0.176777,0.031009,0.062500,0.263822,0.226134,0.125000,0.237171,0.204124
140,0.287837,0.029670,1.000000,0.126872,0.171297,0.253745,0.165274,0.118678,0.112588,0.083918,...,0.251754,0.337068,0.083918,0.147202,0.267026,0.286263,0.035783,0.178017,0.225176,0.290701
150,0.194461,0.066815,0.126872,1.000000,0.385758,0.071429,0.046524,0.267261,0.084515,0.188982,...,0.283473,0.276026,0.188982,0.033150,0.066815,0.040291,0.080582,0.267261,0.084515,0.218218
155,0.350070,0.144338,0.171297,0.385758,1.000000,0.385758,0.251259,0.144338,0.182574,0.204124,...,0.408248,0.298142,0.204124,0.179029,0.216506,0.348155,0.087039,0.144338,0.182574,0.353553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206104,0.329073,0.263822,0.286263,0.040291,0.348155,0.684950,0.262432,0.075378,0.333712,0.426401,...,0.426401,0.194625,0.426401,0.430076,0.640712,1.000000,0.227273,0.150756,0.619751,0.553912
206165,0.219382,0.226134,0.035783,0.080582,0.087039,0.161165,0.314918,0.452267,0.286039,0.426401,...,0.319801,0.155700,0.426401,0.373979,0.376889,0.227273,1.000000,0.150756,0.190693,0.307729
206206,0.121268,0.125000,0.178017,0.267261,0.144338,0.133631,0.174078,0.500000,0.158114,0.353553,...,0.176777,0.258199,0.353553,0.124035,0.125000,0.150756,0.150756,1.000000,0.158114,0.204124


In [75]:
# manhattan_dists = pd.DataFrame(manhattan_distances(high_volume_users),index=high_volume_users.index, columns=high_volume_users.index)
# cosine_dists.head()

user_id_x,7,46,68,137,140,150,155,226,396,417,...,206025,206038,206103,206104,206105,206165,206193,206206,206207,206208
user_id_x,,,,,,,,,,,,,,,,,,,,,
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
46,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
68,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
137,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
140,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.176777,0.0,...,0.204124,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0


In [30]:
cosine_dists = pd.DataFrame(cosine_similarity(high_volume_users),index=high_volume_users.index, columns=high_volume_users.index)
cosine_dists.head()

user_id,7,137,140,150,155,226,396,417,587,593,...,205619,205695,205700,205811,206103,206104,206165,206206,206207,206208
user_id,,,,,,,,,,,,,,,,,,,,,
7,1.000000,0.060634,0.287837,0.194461,0.350070,0.453743,0.253320,0.121268,0.230089,0.171499,...,0.600245,0.375735,0.171499,0.240663,0.545705,0.329073,0.219382,0.121268,0.383482,0.544581
137,0.060634,1.000000,0.029670,0.066815,0.144338,0.200446,0.130558,0.125000,0.316228,0.176777,...,0.088388,0.193649,0.176777,0.031009,0.062500,0.263822,0.226134,0.125000,0.237171,0.204124
140,0.287837,0.029670,1.000000,0.126872,0.171297,0.253745,0.165274,0.118678,0.112588,0.083918,...,0.251754,0.337068,0.083918,0.147202,0.267026,0.286263,0.035783,0.178017,0.225176,0.290701
150,0.194461,0.066815,0.126872,1.000000,0.385758,0.071429,0.046524,0.267261,0.084515,0.188982,...,0.283473,0.276026,0.188982,0.033150,0.066815,0.040291,0.080582,0.267261,0.084515,0.218218
155,0.350070,0.144338,0.171297,0.385758,1.000000,0.385758,0.251259,0.144338,0.182574,0.204124,...,0.408248,0.298142,0.204124,0.179029,0.216506,0.348155,0.087039,0.144338,0.182574,0.353553


In [31]:
# function to enter a user_id and return a list of 10 recommended items
def aisle_recommender(user_id):
    user = high_volume.groupby(['user_id','aisle']).size().sort_values(ascending=False).unstack().fillna(0)
    user_similiartiy = pd.DataFrame(cosine_similarity(user), index=user.index, columns=user.index)

    products = high_volume.groupby(['aisle','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommended = pd.Series(np.dot(products.values,cosine_dists[user_id]), index=products.index)
    return recommended.sort_values(ascending=False)[0:11]

In [32]:
aisle_recommender(155)

aisle
fresh fruits                  1476.865484
fresh vegetables              1184.885867
eggs                           854.879295
packaged vegetables fruits     657.768770
yogurt                         453.978514
packaged cheese                354.282999
milk                           327.339600
soy lactosefree                279.955012
bread                          254.026266
frozen produce                 237.525566
refrigerated                   231.010705
dtype: float64

In [60]:
# function to enter a user_id and return a list of 10 recommended items
def aisle_recommender_tier2(user_id):
    user = high_volume.groupby(['user_id','aisle']).size().sort_values(ascending=False).unstack().fillna(0)
    user_similiartiy = pd.DataFrame(cosine_similarity(user), index=user.index, columns=user.index)

    products = high_volume.groupby(['aisle','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommended = pd.Series(np.dot(products.values,cosine_dists[user_id]), index=products.index)
    return pd.DataFrame(recommended.sort_values(ascending=False)[22:31]).columns

In [43]:
user_155 = master_df[master_df['user_id'] == 155]#.groupby('order_id');

In [45]:
user_155.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'aisle_id', 'department_id', 'department', 'aisle',
       'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'volume_higher'],
      dtype='object')

In [64]:
sns.barplot(user_155['aisle'].values)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [61]:
aisle_recommender_tier2(155)

,0
aisle,
fresh herbs,113.860007
other creams cheeses,109.800892
baby food formula,109.665896
soup broth bouillon,109.472238
butter,104.381195
nuts seeds dried fruit,102.115394
spreads,99.802153
canned jarred vegetables,99.216408
frozen meals,98.144236


In [63]:
product_recommender(155)

product_name
Organic Egg Whites                   30.202771
100% Raw Coconut Water                6.140192
Organic Baby Spinach                  3.951781
Banana                                3.699367
Bag of Organic Bananas                3.558407
Organic Hass Avocado                  2.550437
Organic Strawberries                  2.546313
Organic Raspberries                   1.855532
Organic Grape Tomatoes                1.812495
Organic Zucchini                      1.711898
Organic SprouTofu Super Firm Tofu     1.584164
dtype: float64

In [64]:
product_recommender(206206)

product_name
Large Alfresco Eggs       66.693176
Bag of Organic Bananas     8.485447
Organic Baby Spinach       5.484864
Organic Strawberries       5.418419
Banana                     4.719492
Organic Hass Avocado       4.704853
Organic Avocado            3.483861
Organic Yellow Onion       3.144441
Organic Whole Milk         3.097685
Organic Whole Cashews      2.983461
Organic Zucchini           2.392403
dtype: float64

In [66]:
users = high_volume.user_id_x.unique().tolist()

# calculate recall for 2500 users
def recommender_quality():
    recs = []
    for user in sorted(users)[:1000]:
        recommendations = product_recommender(user)
        top_20_products = [user_id == user].product_name.value_counts().head(20)
    
        recommendations_list = recommendations.index.tolist()
        top_20_items_list = top_20_products.index.tolist()
    
        res.append((len(set(recommendations_list) & set(top_20_items_list)))/5)
    return np.mean(res)
# get metric for the :1000 users
recommender_quality

<function __main__.recommender_quality()>

In [67]:
recommender_corr_df[206206].sort_values()[1:11]
#recommender_2_df['Matrix, The (1999)'].sort_values()[1:11] # top 10 movie recommendations for the matrix

user_id_x
140437    0.292947
144972    0.292947
66804     0.292947
40375     0.292947
201026    0.292947
47087     0.292947
89799     0.292947
151750    0.292947
18640     0.292947
59879     0.422796
Name: 206206, dtype: float64

In [68]:
def product_recommender_corr(user_id):
    user = high_volume.groupby(['user_id_x','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    user_similiartiy_corr = pd.DataFrame(pairwise_distances(user, metric='correlation'), index=user.index, columns=user.index)

    products = high_volume.groupby(['product_name','user_id_x']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommended = pd.Series(np.dot(products.values,cosine_dists[user_id]), index=products.index)
    return recommended.sort_values(ascending=False)[0:11]

In [69]:
product_recommender_corr(206206)

product_name
Large Alfresco Eggs       66.693176
Bag of Organic Bananas     8.485447
Organic Baby Spinach       5.484864
Organic Strawberries       5.418419
Banana                     4.719492
Organic Hass Avocado       4.704853
Organic Avocado            3.483861
Organic Yellow Onion       3.144441
Organic Whole Milk         3.097685
Organic Whole Cashews      2.983461
Organic Zucchini           2.392403
dtype: float64

In [70]:
def product_recommender_euclidean(user_id):
    user = high_volume.groupby(['user_id_x','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    user_similiartiy_corr = pd.DataFrame(pairwise_distances(user, metric='euclidean'), index=user.index, columns=user.index)

    products = high_volume.groupby(['product_name','user_id_x']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommended = pd.Series(np.dot(products.values,cosine_dists[user_id]), index=products.index)
    return recommended.sort_values(ascending=False)[0:11]

In [71]:
product_recommender_euclidean(206206)

product_name
Large Alfresco Eggs       66.693176
Bag of Organic Bananas     8.485447
Organic Baby Spinach       5.484864
Organic Strawberries       5.418419
Banana                     4.719492
Organic Hass Avocado       4.704853
Organic Avocado            3.483861
Organic Yellow Onion       3.144441
Organic Whole Milk         3.097685
Organic Whole Cashews      2.983461
Organic Zucchini           2.392403
dtype: float64

In [76]:
def product_recommender_manhattan(user_id):
    user = high_volume.groupby(['user_id_x','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    user_similiartiy_corr = pd.DataFrame(pairwise_distances(user, metric='manhattan'), index=user.index, columns=user.index)

    products = high_volume.groupby(['product_name','user_id_x']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommended = pd.Series(np.dot(products.values,manhattan_dists[user_id]), index=products.index)
    return recommended.sort_values(ascending=False)[0:11]

In [77]:
product_recommender_manhattan(206206)

product_name
Banana                                         5227.0
Bag of Organic Bananas                         4629.0
Organic Hass Avocado                           3044.0
Organic Baby Spinach                           2921.0
Organic Strawberries                           2803.0
Organic Raspberries                            2277.0
Organic Grade A Free Range Large Brown Eggs    1966.0
Organic Avocado                                1808.0
Organic Large Brown Grade AA Cage Free Eggs    1778.0
Organic Whole Milk                             1714.0
Limes                                          1464.0
dtype: float64